In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('day6')\
        .config('spark.driver.extraClassPath','/usr/lib/jvm/java-11-openjdk-amd64/lib/postgresql-42.6.0.jar')\
        .getOrCreate()






Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/11 16:52:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/11 16:52:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/11 16:52:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
from pyspark.sql.functions import col, cast,concat_ws,substring,lit,to_date,udf, when,length, expr
# loading the revised_final dataset from the data folder 


In [3]:


data_path = "./data/revised_final.csv"  
df = spark.read.csv(data_path, header=True, inferSchema=True)
#loading goods_classifiation 
data_path = "./data/goods_classification.csv"  
df_goods = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./data/country_classification.csv"  
df_country = spark.read.csv(data_path, header=True, inferSchema=True)
data_path = "./data/services_classification.csv"  
df_services = spark.read.csv(data_path, header=True, inferSchema=True)


In [4]:

df.show()
df.printSchema()
#number of rows 
df.count()
#we haveto change the time_ref into string so that we can use string methods for manipulation. 
df = df.withColumn("time_ref", col("time_ref").cast("string"))


#you can see that the time_ref has been changed into string 
df.printSchema()
#concat_ws is used here to concat strings with delimiter "\"
#lit is used to append "30" as  a string at theend of the data.
df = df.withColumn("time_ref", concat_ws("/", substring(col("time_ref"), 1, 4), substring(col("time_ref"), 5, 2), lit('30')))

df.show()
df.printSchema()
#now to change the string type column to date format. we use to_date
df = df.withColumn("time_ref", to_date(col("time_ref"), "yyyy/MM/dd"))

df.printSchema()
null_count_accnt = df.filter(col("product_type").isNull()).count()
null_count_code = df.filter(col("product_type").isNull()).count()
null_count_country_code = df.filter(col("product_type").isNull()).count()
null_count_product_type = df.filter(col("product_type").isNull()).count()
null_count_value = df.filter(col("product_type").isNull()).count()
null_count_status = df.filter(col("status").isNull()).count()



+--------+-------+----+------------+------------+-------------+------+
|time_ref|account|code|country_code|product_type|        value|status|
+--------+-------+----+------------+------------+-------------+------+
|  202306|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|  202306|Exports|  00|          AG|       Goods|     351919.0|     F|
|  202306|Exports|  00|          AI|       Goods|      84762.0|     F|
|  202306|Exports|  00|          AL|       Goods|       3463.0|     F|
|  202306|Exports|  00|          AM|       Goods|     679586.0|     F|
|  202306|Exports|  00|          AO|       Goods|     464583.0|     F|
|  202306|Exports|  00|          AR|       Goods|    5943055.0|     F|
|  202306|Exports|  00|          AS|       Goods|  1.1622992E7|     F|
|  202306|Exports|  00|          AT|       Goods|  1.1202334E7|     F|
|  202306|Exports|  00|          AU|       Goods|2.272665701E9|     F|
|  202306|Exports|  00|          AW|       Goods|     362396.0|     F|
|  202

root
 |-- time_ref: string (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)

+----------+-------+----+------------+------------+-------------+------+
|  time_ref|account|code|country_code|product_type|        value|status|
+----------+-------+----+------------+------------+-------------+------+
|2023/06/30|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|2023/06/30|Exports|  00|          AG|       Goods|     351919.0|     F|
|2023/06/30|Exports|  00|          AI|       Goods|      84762.0|     F|
|2023/06/30|Exports|  00|          AL|       Goods|       3463.0|     F|
|2023/06/30|Exports|  00|          AM|       Goods|     679586.0|     F|
|2023/06/30|Exports|  00|          AO|       Goods|     464583.0|     F|
|2023/06/30|Exports|  00|          AR|       Goods|    5943

In [5]:

print(null_count_accnt)
print(null_count_code)
print(null_count_country_code)
print(null_count_product_type)
print(null_count_status)
print(null_count_value)



0
0
0
0
0
0


In [6]:


df_goods.printSchema()
df_goods.show()
df_services.printSchema()
df_services.show()
df_country.printSchema()
df_country.show()
# here we are dropping the rows which has TOT and other unnecessary total named rows in the dataframe for the ease of analysis


df = df.filter(df['country_code'] != 'TOT')

df = df.filter(df['country_code'] != 'TOT (BoP basis)')

df = df.filter(df['country_code'] != 'TOT (OMT FOB)')
df = df.filter(df['country_code'] != 'TOT (OMT CIF)')
df = df.filter(df['country_code'] != 'TOT (OMT VFD)')



df = df.withColumn("code",
    when((col("product_type") == "Goods") & (length(col("code")) == 4), substring(col("code"), 1, 2))
    .when((col("product_type") == "Goods") & (length(col("code")) == 3), substring(col("code"), 1, 1))
    .otherwise(col("code")))

# Show the modified DataFrame
df.show(500)
# writing the clean dasta into the data folder 
# after writing the file , the default format that the write method uses is parquet format which is faster to lead and perform operations so we use the parquet file format



root
 |-- Level_1: integer (nullable = true)
 |-- Level_1_desc: string (nullable = true)

+-------+--------------------+
|Level_1|        Level_1_desc|
+-------+--------------------+
|      1|       Animals; live|
|      1|       Animals; live|
|      1|       Animals; live|
|      1|       Animals; live|
|      1|       Animals; live|
|      1|       Animals; live|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      2|Meat and edible m...|
|      3|Fish and crustace...|
|      3|Fish and crustace...|
|      3|Fish and crustace...|
|      3|Fish and crustace...|
+-------+--------------------+
only showing top 20 rows

root
 |-- code: string (nullable = true)
 |-- service_label: string (nullable = true)

+-------+--------------------+
|   code|       servic

In [7]:
#renaming columns of df_goods

df_goods = df_goods.withColumnRenamed('Level_1','code')\
        .withColumnRenamed('Level_1_desc','goods_category')

In [8]:
#writing in parquet file 
df.write.format("parquet")\
    .mode('overwrite')\
    .options(compression="snappy")\
    .save("./data_cleaned/revised_final_cleaned/cleaned_revised.parquet")



df_goods.coalesce(1).write.format("parquet").mode("overwrite").\
options(compression='snappy').\
save("./data_cleaned/goods_cleaned/cleaned_goods.parquet")
df_services.coalesce(1).write.format("parquet").mode("overwrite").\
options(compression='snappy').\
save("./data_cleaned/services_cleaned/cleaned_sevices.parquet")
df_country.coalesce(1).write.format("parquet").mode("overwrite").\
options(compression='snappy').\
save("./data_cleaned/countries_cleaned/cleaned_countries.parquet")






23/09/11 16:52:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [9]:
#writing the main table to the postgres database
df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'cleaned_data', user='postgres',password='postgres').mode('overwrite').save()


In [10]:
#writing data to the table in prostgres 

df_goods.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'cleaned_goods', user='postgres',password='postgres').mode('overwrite').save()
df_services.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'cleaned_services', user='postgres',password='postgres').mode('overwrite').save()
df_country.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'cleaned_countries', user='postgres',password='postgres').mode('overwrite').save()


In [11]:
df_goods.show()

+----+--------------------+
|code|      goods_category|
+----+--------------------+
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   3|Fish and crustace...|
|   3|Fish and crustace...|
|   3|Fish and crustace...|
|   3|Fish and crustace...|
+----+--------------------+
only showing top 20 rows

